In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [2]:
# Load the data
data = pd.read_csv('epl-training.csv')
pd.set_option('display.max_columns',None)
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,19/08/2000,Charlton,Man City,4,0,H,2,0,H,Rob Harris,17,8,14,4,6,6,13,12,1,2,0,0
1,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0
2,19/08/2000,Coventry,Middlesbrough,1,3,A,1,1,D,Barry Knight,6,16,3,9,8,4,15,21,5,3,1,0
3,19/08/2000,Derby,Southampton,2,2,D,1,2,A,Andy D'Urso,6,13,4,6,5,8,11,13,1,1,0,0
4,19/08/2000,Leeds,Everton,2,0,H,2,0,H,Dermot Gallagher,17,12,8,6,6,4,21,20,1,3,0,0


In [37]:
# Team Names
le_teams = LabelEncoder()

# Using the same label encoder for the home and away teams.
le_teams.fit(pd.concat([data['HomeTeam'],data['AwayTeam']])) 

# Using the label encoder to encode the home and away teams
data['HomeTeam_Encoded'] = le_teams.transform(data['HomeTeam'])
data['AwayTeam_Encoded'] = le_teams.transform(data['AwayTeam'])

# FTR and HTR
le_results = LabelEncoder()
le_results.fit(pd.concat([data['FTR'],data['HTR']]))

# Using the label encoder to encode the FTR and HTR
data['FTR_Encoded'] = le_results.transform(data['FTR'])
data['HTR_Encoded'] = le_results.transform(data['HTR'])

# Referee
le_referee = LabelEncoder()
le_referee.fit(data['Referee'])

data['Referee_Encoded'] = le_referee.transform(data['Referee'])
# Team Names
le_teams = LabelEncoder()

data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HomeTeam_Encoded,AwayTeam_Encoded,FTR_Encoded,HTR_Encoded,Referee_Encoded,nFTHG_Pairwise,nHTHG_Pairwise,nHS_Pairwise,nHC_Pairwise,nHF_Pairwise,nHY_Pairwise,nHR_Pairwise,nFTAG_Pairwise,nHTAG_Pairwise,nAS_Pairwise,nAC_Pairwise,nAF_Pairwise,nAY_Pairwise,nAR_Pairwise
0,19/08/2000,Charlton,Man City,4,0,H,2,0,H,Rob Harris,17,8,14,4,6,6,13,12,1,2,0,0,12,26,2,2,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,13,43,2,2,65,1.000000,0.000000,16.000000,7.000000,12.000000,1.000000,0.0,0.00,0.000000,8.000000,4.000000,20.0,4.000000,1.000000
2,19/08/2000,Coventry,Middlesbrough,1,3,A,1,1,D,Barry Knight,6,16,3,9,8,4,15,21,5,3,1,0,14,28,0,1,19,2.000000,0.000000,15.500000,8.000000,15.000000,2.500000,0.0,1.00,1.000000,8.000000,3.500000,17.0,2.000000,0.500000
3,19/08/2000,Derby,Southampton,2,2,D,1,2,A,Andy D'Urso,6,13,4,6,5,8,11,13,1,1,0,0,16,36,1,0,12,2.333333,0.333333,14.666667,7.333333,13.666667,2.333333,0.0,1.00,0.666667,9.333333,5.333333,18.0,2.333333,0.333333
4,19/08/2000,Leeds,Everton,2,0,H,2,0,H,Dermot Gallagher,17,12,8,6,6,4,21,20,1,3,0,0,22,17,2,2,46,2.250000,0.250000,14.500000,6.000000,14.250000,2.500000,0.0,0.75,0.500000,8.500000,5.500000,17.5,2.750000,0.250000


In [38]:
# Number of matches to look back
n = 5

# Precompute rolling averages for all attributes
def compute_pairwise_rolling(data, attributes, n):
    for attribute in attributes:
        # Group by HomeTeam and AwayTeam
        grouped = data.groupby(['HomeTeam', 'AwayTeam'])[attribute]
        # Calculate rolling averages and align the index
        rolling_avg = grouped.apply(lambda x: x.shift(1).rolling(window=n, min_periods=1).mean())
        # Align the new column with the original DataFrame's index
        data[f"n{attribute}_Pairwise"] = rolling_avg.values
    return data

# List of attributes to calculate rolling averages for
attributes = ['FTHG', 'HTHG', 'HS', 'HC', 'HF', 'HY', 'HR',
              'FTAG', 'HTAG', 'AS', 'AC', 'AF', 'AY', 'AR']

# Apply the optimized function
data = compute_pairwise_rolling(data, attributes, n)

display(data)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HomeTeam_Encoded,AwayTeam_Encoded,FTR_Encoded,HTR_Encoded,Referee_Encoded,nFTHG_Pairwise,nHTHG_Pairwise,nHS_Pairwise,nHC_Pairwise,nHF_Pairwise,nHY_Pairwise,nHR_Pairwise,nFTAG_Pairwise,nHTAG_Pairwise,nAS_Pairwise,nAC_Pairwise,nAF_Pairwise,nAY_Pairwise,nAR_Pairwise
0,19/08/2000,Charlton,Man City,4,0,H,2,0,H,Rob Harris,17,8,14,4,6,6,13,12,1,2,0,0,12,26,2,2,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,13,43,2,2,65,1.000000,0.000000,16.000000,7.000000,12.000000,1.000000,0.0,0.00,0.000000,8.000000,4.000000,20.0,4.000000,1.000000
2,19/08/2000,Coventry,Middlesbrough,1,3,A,1,1,D,Barry Knight,6,16,3,9,8,4,15,21,5,3,1,0,14,28,0,1,19,2.000000,0.000000,15.500000,8.000000,15.000000,2.500000,0.0,1.00,1.000000,8.000000,3.500000,17.0,2.000000,0.500000
3,19/08/2000,Derby,Southampton,2,2,D,1,2,A,Andy D'Urso,6,13,4,6,5,8,11,13,1,1,0,0,16,36,1,0,12,2.333333,0.333333,14.666667,7.333333,13.666667,2.333333,0.0,1.00,0.666667,9.333333,5.333333,18.0,2.333333,0.333333
4,19/08/2000,Leeds,Everton,2,0,H,2,0,H,Dermot Gallagher,17,12,8,6,6,4,21,20,1,3,0,0,22,17,2,2,46,2.250000,0.250000,14.500000,6.000000,14.250000,2.500000,0.0,0.75,0.500000,8.500000,5.500000,17.5,2.750000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9215,19/05/2024,Crystal Palace,Aston Villa,5,0,H,2,0,H,D Bond,15,8,9,2,10,8,2,4,1,4,0,0,15,1,2,2,31,1.800000,0.600000,15.400000,5.800000,8.200000,1.400000,0.0,0.80,0.600000,7.200000,2.600000,9.0,1.800000,0.000000
9216,19/05/2024,Liverpool,Wolves,2,0,H,2,0,H,C Kavanagh,36,4,14,3,14,11,10,2,1,1,0,1,24,45,2,2,23,1.800000,0.400000,17.000000,5.000000,8.200000,1.400000,0.0,0.60,0.600000,8.600000,2.600000,8.8,2.000000,0.000000
9217,19/05/2024,Luton,Fulham,2,4,A,1,2,A,M Donohue,15,15,6,7,15,20,4,4,5,4,0,0,25,18,0,0,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9218,19/05/2024,Man City,West Ham,3,1,H,2,1,H,J Brooks,28,3,12,2,3,12,11,2,0,1,0,0,26,43,2,2,71,0.000000,0.000000,9.000000,4.000000,14.000000,4.000000,1.0,2.00,0.000000,20.000000,3.000000,16.0,3.000000,1.000000


In [40]:
input_features = ['HomeTeam_Encoded','AwayTeam_Encoded']
data_input = data[input_features]
display(data_input)

,HomeTeam_Encoded,AwayTeam_Encoded
0,12,26
1,13,43
2,14,28
3,16,36
4,22,17
...,...,...
9215,15,1
9216,24,45
9217,25,18
9218,26,43


In [41]:
output_features = ['FTR_Encoded']
data_output = data['FTR_Encoded']
display(data_output)

0       2
1       2
2       0
3       1
4       2
       ..
9215    2
9216    2
9217    0
9218    2
9219    0
Name: FTR_Encoded, Length: 9220, dtype: int32

In [42]:
# Features and target
X = data[input_features]
y = data['FTR_Encoded']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

# Predictions
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))



Accuracy: 0.39566217548471905
Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.44      0.39       869
           1       0.27      0.32      0.29       761
           2       0.55      0.41      0.47      1413

    accuracy                           0.40      3043
   macro avg       0.39      0.39      0.38      3043
weighted avg       0.42      0.40      0.40      3043



In [53]:
# Fit the LabelEncoder during preprocessing
le_teams = LabelEncoder()
le_teams.fit(pd.concat([data['HomeTeam'], data['AwayTeam']]))

def predict_match(home_team, away_team):
    # Encode the team names
    home_team_encoded = le_teams.transform([home_team])[0]
    away_team_encoded = le_teams.transform([away_team])[0]

    # Prepare the feature vector
    match_features = pd.DataFrame([[home_team_encoded, away_team_encoded]], 
                                   columns=['HomeTeam_Encoded', 'AwayTeam_Encoded'])
    
    # Scale the features
    match_features_scaled = scaler.transform(match_features)

    # Predict the outcome
    predicted_result = svm_model.predict(match_features_scaled)

    # Map encoded result to human-readable outcome
    result_mapping = {0: "Draw", 1: "Home Win", 2: "Away Win"}
    return result_mapping[predicted_result[0]]

# Test Prediction
home_team = 'Man United'
away_team = 'Man City'
outcome = predict_match(home_team, away_team)
print(f"The predicted outcome for {home_team} vs {away_team} is: {outcome}")



The predicted outcome for Man United vs Man City is: Away Win
